In [1]:
##importing required libraries
import pandas as pd
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import os
import random 
import kaggle
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import random

In [2]:
#Downloading Data
# Set the Kaggle dataset and file details
kaggle_dataset = 'deadskull7/fer2013'  # Example: 'zillow/zecon'
file_name = 'fer2013.csv'  # Name of the file you want to download
data_folder = '../data'

# Create data directory if it does not exist
if not os.path.exists(data_folder):
    os.makedirs(data_folder)

try:
    # Download the dataset file from Kaggle
    kaggle.api.dataset_download_file(dataset=kaggle_dataset, file_name=file_name, path=data_folder)
    # Define the path to the downloaded zip file
    zip_file_path = os.path.join(data_folder, file_name+'.zip')
    
    # Unzip the file
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(data_folder)

    # Remove the zip file after extraction
    os.remove(zip_file_path)
    print('Data downloaded and extracted.')

except Exception as e:
    print(f"An error occurred: {e}")

Dataset URL: https://www.kaggle.com/datasets/deadskull7/fer2013
Data downloaded and extracted.


In [3]:
##import the dataset
faces = pd.read_csv('../data/fer2013.csv')
faces.head()


,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training
